<a href="https://colab.research.google.com/github/anjali-ojha/deep-learning/blob/a11/assignment/assignment-11/Anjali_Ojha_HW_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GitHub Link - https://github.com/anjali-ojha/deep-learning/blob/a11/assignment/assignment-11/Anjali_Ojha_HW_11.ipynb

# Write prompts using OPENAI API keys that perform the following task and display the output:

1. Write a chatbot prompt to iteratively create a sequence of chats on one particular topic.

    a. Ask the bot to solve one complex math problem.
    
    b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website.

    c. At the end, ask the bot to summarize your chat

In [ ]:
# ! pip install openai==0.28
# ! pip install python-dotenv

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def ask_gpt(prompt, model="gpt-3.5-turbo"):
    """
    This method will make a API call to the OPEN-AI server and get the response.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

# a. Ask the bot to solve one complex math problem.

Asking GPT to solve a quadratic equation.

6x² + 11x - 35 = 0

In [ ]:
maths_questions = f""" solve the equation 6x² + 11x - 35 = 0 """

maths_questions_response = ask_gpt(maths_questions)
print(f"User : {maths_questions}")
print(f"\n\nAssistant : {maths_questions_response}")

User :  solve the equation 6x² + 11x - 35 = 0 


Assistant : To solve the equation 6x² + 11x - 35 = 0, we can use the quadratic formula:

x = (-b ± √(b² - 4ac)) / 2a

In this case, a = 6, b = 11, and c = -35. Plugging these values into the formula, we get:

x = (-11 ± √(11² - 4*6*(-35))) / 2*6
x = (-11 ± √(121 + 840)) / 12
x = (-11 ± √961) / 12
x = (-11 ± 31) / 12

This gives us two possible solutions:

x₁ = (-11 + 31) / 12 = 20 / 12 = 5/3
x₂ = (-11 - 31) / 12 = -42 / 12 = -7/2

Therefore, the solutions to the equation 6x² + 11x - 35 = 0 are x = 5/3 and x = -7/2.


# b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website.

In [ ]:
! pip install PyPDF2

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_website_text(website_url):
    """ This method will read the webpage fromn the link and return the content of it."""
    response = requests.get(website_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    website_text = '\n'.join([p.get_text() for p in soup.find_all('p')])
    return website_text

Asking GPT to read a Medium article and answer the question based on that. Link to the article is https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6

In [ ]:
website_questions  = f"""1 - Where is Yosemite located?, 2 - What are some iconic places in Yosemite National Park?"""

website_data = extract_website_text("https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6")[:5000]

website_questions_prompt = f"""
Answer following questions from the given Text:
{website_questions}

Separate your answers with line breaks. First put question then the corresponding answer.

website data:
```{website_data}```
"""
website_questions_response = ask_gpt(website_questions_prompt)
print(f"User : {website_questions}")
print(f"\n\nAssistant : {website_questions_response}")

User : 1 - Where is Yosemite located?, 2 - What are some iconic places in Yosemite National Park?


Assistant : 1 - Where is Yosemite located?
Yosemite National Park is located within the Sierra Nevada Mountains of California.

2 - What are some iconic places in Yosemite National Park?
Some iconic places in Yosemite National Park include El Capitan, Half Dome, Bridalveil Fall, Yosemite Falls, Vernal and Nevada Falls, Ribbon Fall, Glacier Point, Sentinel Dome, and the Mist Trail.


In [ ]:
from PyPDF2 import PdfReader


def extract_pdf_text(pdf_url):
    """ This method will read a pdf file fromn the link and return the content of it."""
    response = requests.get(pdf_url)
    with open('temp.pdf', 'wb') as f:
        f.write(response.content)

    pdf_text = ''
    with open('temp.pdf', 'rb') as pdf_file:
        reader = PdfReader(pdf_file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            pdf_text += page.extract_text()

    return pdf_text

Asking GPT to read a PDF file context and then asnwer question from it. Link to the article is https://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf

In [ ]:
document_questions  = f"""3 - What is spark ?, 4 - How spark works ? """

pdf_data = extract_pdf_text("https://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf")[:5000]

document_questions_prompt = f"""
Answer following questions from the given Text:
{document_questions}

Separate your answers with line breaks. First put question then the corresponding answer.

Data from PDF file:
```{pdf_data}```
"""
document_questions_response = ask_gpt(document_questions_prompt)
print(f"User : {document_questions}")
print(f"\n\nAssistant : {document_questions_response}")

User : 3 - What is spark ?, 4 - How spark works ? 


Assistant : 3 - What is spark ?
Apache Spark is a unified computing engine and a set of libraries for parallel data processing on computer clusters.

4 - How spark works ?
Spark supports multiple widely used programming languages (Python, Java, Scala and R), includes libraries for diverse tasks ranging from SQL to streaming and machine learning, and runs anywhere from a laptop to a cluster of thousands of servers. This makes it an easy system to start with and scale up to big data processing or incredibly large scale.


# c. At the end, ask the bot to summarize your chat

In [ ]:
summary_question = f""" Summarize all the above chats I did with the openai apis. Here are all the interations in details having one line for each question and summzarize answers. Make the overall reponse 500 characters long. Give the reponse in differnet points seperated by line breaks

```User - {maths_questions}```
```Assistant - {maths_questions_response}```

```User - {website_questions_prompt}```
```Assistant - {website_questions_response}```

```User - {document_questions_prompt}```
```Assistant - {document_questions_response}```

"""

ques = summary_question.split('\n')[0]
print(f"User : {ques}")
print(f"\n\nAssistant : ")

response = ask_gpt(summary_question)
for line in response.split("\n"):
    print(line)


User :  Summarize all the above chats I did with the openai apis. Here are all the interations in details having one line for each question and summzarize answers. Make the overall reponse 500 characters long. Give the reponse in differnet points seperated by line breaks


Assistant : 
1. To solve the equation 6x² + 11x - 35 = 0, we use the quadratic formula to find x = 5/3 and x = -7/2.
2. Yosemite National Park is located in the Sierra Nevada Mountains of California.
3. Some iconic places in Yosemite National Park include El Capitan, Half Dome, Bridalveil Fall, and more.
4. Apache Spark is a unified computing engine and set of libraries for parallel data processing on computer clusters.
5. Spark supports multiple programming languages and diverse tasks, making it easy to start with and scale up for big data processing.


# Converting the above fragmented Questions and Answers in the chat app format.

# Chat APP

In [ ]:
def ask_gpt_with_context(prompt, model="gpt-3.5-turbo", context=None):
    """
    This method talk to OPEN-AI server and also update the context with the
    communication and keep all the communication and context at one place, which
    can be later used for  queries.
    """

    messages = []

    if context:
        messages.extend(context)

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )

    return response.choices[0].message["content"], messages


## List of questions we are going to ask from the gpt -

* solve the equation 6x² + 11x - 35 = 0

* Load data from website - https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6
    
    * where is yosemite located?
    
    * what are the popular attractions there?

* Load data from the pdf file - https://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf
    
    * what is spark?
    
    * what are advanatages of spark?



In [ ]:
questions = [
    "solve the equation 6x² + 11x - 35 = 0",
    "Load data from website - https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6",
    "where is yosemite located?",
    "what are the popular attractions there?",
    "Load data from the pdf file - https://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf",
    "what is spark?",
    "what are advanatages of spark?",
    "summarize the whole chat history including all details from the start of the conversation and make the list pointwise"
]

In [ ]:
# Running a sequence of the questions though the GPT server.
import time

context = None


for question in questions:
    print(f"\nUser : {question}")
    question = question.strip()

    if question.lower().startswith("load"):

        extra_data = ""

        if question.lower().endswith("pdf"):
            pdf_url = [x for x in question.split(" ") if x.startswith("https://")][0]
            pdf_text = extract_pdf_text(pdf_url)
            extra_data = pdf_text[:6000]

        else:
            web_url = [x for x in question.split(" ") if x.startswith("https://")][0]
            website_data = extract_website_text(web_url)
            extra_data = website_data[:5000]

        question = f""" remember the data

        ```{extra_data}```
        """

    else:
        question = question

    response, new_context = ask_gpt_with_context(question, context=context)
    new_context.append({"role": "assistant", "content" : response})
    context = new_context

    print(f"\nAssistant: {response}")
    print("\n------------------------------------\n")
    time.sleep(3)



User : solve the equation 6x² + 11x - 35 = 0

Assistant: To solve the equation 6x² + 11x - 35 = 0, we can use the quadratic formula:

x = (-b ± √(b² - 4ac)) / 2a

In this case, a = 6, b = 11, and c = -35. Plugging these values into the formula, we get:

x = (-11 ± √(11² - 4*6*(-35))) / 2*6
x = (-11 ± √(121 + 840)) / 12
x = (-11 ± √961) / 12
x = (-11 ± 31) / 12

This gives us two possible solutions:

x₁ = (-11 + 31) / 12 = 20 / 12 = 5/3
x₂ = (-11 - 31) / 12 = -42 / 12 = -7/2

Therefore, the solutions to the equation 6x² + 11x - 35 = 0 are x = 5/3 and x = -7/2.

------------------------------------


User : Load data from website - https://medium.com/@vdwayne/yosemite-national-park-a-natural-marvel-with-a-ton-of-wasted-space-626b1f185df6

Assistant: I'm sorry, I'm not sure what you're asking for in relation to the provided text about Yosemite National Park. Could you please clarify your question or let me know how I can assist you further?

------------------------------------


User : 

## Observations -

  * As we can see that, we first run a question at a time and get the response.

  * Later we automate the entire questions through a series of inputs.

  * We also keep the context with all the communications and later can reference it, so that system knows in each communication what topic we are talking about.

  * And we can see that it also generated the communication summary.